In [ ]:
# !pip install gpytorch==1.14
# !pip install botorch==0.13.0

In [ ]:
%matplotlib widget
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import pymongo
import os
import sys
import re
import time
import yaml
import json
import botorch
from botorch.models.transforms.input import AffineInputTransform
from botorch.models import MultiTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.kernels import ScaleKernel, RBFKernel
from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize

import gpytorch
from gpytorch.mlls import ExactMarginalLogLikelihood

In [ ]:
print("BoTorch version:", botorch.__version__)
print("GPyTorch version:", gpytorch.__version__)

In [ ]:
# Select experimental setup for which we are training a model
setup = "ip2"

In [ ]:
# Open credential file for database
with open(os.path.join(os.getenv('HOME'), 'db.profile')) as f:
    db_profile = f.read()

# Connect to the MongoDB database with read-only access
db = pymongo.MongoClient(
    host="mongodb05.nersc.gov",
    username="bella_sf_ro",
    password=re.findall('SF_DB_READONLY_PASSWORD=(.+)', db_profile)[0],
    authSource="bella_sf")["bella_sf"]

# Extract data from the database as pandas dataframe
collection = db[setup]
df = pd.DataFrame( list(collection.find()) )

In [ ]:
# Extract the name of inputs and outputs for this setup
with open("../../dashboard/config/variables.yml") as f:
    yaml_dict = yaml.safe_load( f.read() )
input_variables = yaml_dict[setup]["input_variables"]
input_names = [ v['name'] for v in input_variables.values() ] 
output_variables = yaml_dict[setup]["output_variables"]
output_names = [ v['name'] for v in output_variables.values() ]

In [ ]:
# Visualize the dimensional data
ax = plt.figure().add_subplot(projection='3d')

ax.scatter( 
    df[input_names[0]], 
    df[input_names[-1]], 
    df[output_names[0]], 
    c=df.experiment_flag, 
    alpha=0.3)

ax.view_init(elev=40., azim=40, roll=0)
plt.xlabel(input_names[0])
plt.ylabel(input_names[-1])

# Normalize with affine input/output transformer

In [ ]:
# Define the input and output normalizations
X = torch.tensor( df[ input_names ].values, dtype=torch.float )

# Forward input and output transforms
input_transform = AffineInputTransform( 
    len(input_names), 
    coefficient=X.std(axis=0), 
    offset=X.mean(axis=0)
)

def reverse_affine_transform(x_normalized, transform):
    return transform.coefficient * x_normalized + transform.offset
    
y = torch.tensor( df[ output_names ].values, dtype=torch.float )
output_transform = AffineInputTransform( 
    len(output_names), 
    coefficient=y.std(axis=0),
    offset=y.mean(axis=0)
)

# Reverse output transformations
output_transform_reversed = AffineInputTransform(
    d=len(output_names),
    coefficient=output_transform.coefficient,
    offset=output_transform.offset,
    reverse=True  
)

if (min(X.mean(axis=0)) == 0):
    print("Mean value used for normalization is 0. This will lead to NaNs ",X.mean(axis=0))
if (min(X.std(axis=0)) == 0):
    print("RMS value used for normalization is 0. This will lead to NaNs ", X.std(axis=0))

# Save input/output normalization parameters to JSON for future inference

In [ ]:
import json
normalization_info = {
    "input_mean": input_transform.offset.tolist(),
    "input_std": input_transform.coefficient.tolist(),
    "output_mean": output_transform.offset.tolist(),
    "output_std": output_transform.coefficient.tolist()
}

with open('./normalization/normalization.json', 'w') as f:
    json.dump(normalization_info, f)

# Apply normalization to the data set

In [ ]:
norm_df = df.copy()
norm_df[input_names] = input_transform( torch.tensor( df[input_names].values ) )
norm_df[output_names] = output_transform( torch.tensor( df[output_names].values ) )

# Visualize the normalized data

In [ ]:
ax = plt.figure().add_subplot(projection='3d')

ax.scatter( 
    norm_df[input_names[0]], 
    norm_df[input_names[-1]], 
    norm_df[output_names[0]], 
    c=norm_df.experiment_flag, 
    alpha=0.3)

ax.view_init(elev=40., azim=40, roll=0)
plt.xlabel(input_names[0])
plt.ylabel(input_names[-1])

# Define a multi-input multi-task GP model

In [ ]:
%%time
gp_model = MultiTaskGP(
    torch.tensor( norm_df[['experiment_flag']+input_names].values ),
    torch.tensor( norm_df[output_names].values ),
    task_feature=0,
    covar_module=ScaleKernel(RBFKernel()),
    outcome_transform=None,
)
# Fit the model
mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
fit_gpytorch_mll(mll)

# cov = gp_model.task_covar_module._eval_covar_matrix()
# print( 'Correlation: ', cov[1,0]/torch.sqrt(cov[0,0]*cov[1,1]).item() )

# Construct LUME GPModel 

In [ ]:
!pip uninstall --yes lume-model

In [ ]:
!pip install git+https://github.com/slaclab/lume-model.git@41fd8a5a3144a7bddca39b806500caabe2187262

In [ ]:
from lume_model.variables import ScalarVariable, DistributionVariable
from lume_model.models.gp_model import GPModel

input_variables = [
    ScalarVariable(name="TOD_fs3"),
    ScalarVariable(name="GVD"),
    ScalarVariable(name="z_target_um"),   
]
output_variables = [
    DistributionVariable(name="n_protons_sim_task", distribution_type="MultiVariateNormal"),
    DistributionVariable(name="n_protons_exp_task", distribution_type="MultiVariateNormal")
]

In [ ]:
gp_model.eval()

gp_lume_model = GPModel(
    model=gp_model, input_variables=input_variables, output_variables=output_variables, input_transformers=[], output_transformers=[], jitter=1e-4,
)

# Print and plot normalized predictions by running lume GP model

In [ ]:
# Plot predictions on physical (unnormalized) data
tod_max = norm_df['TOD_fs3'].max()
gvd_fix = (norm_df['GVD'].max()+ norm_df['GVD'].min())/2
zmin, zmax = norm_df['z_target_um'].min(), norm_df['z_target_um'].max()

z_test_array = torch.tensor (np.linspace(zmin,zmax,100).reshape(-1, 1), dtype=torch.float32) 
TOD_test_array  = torch.tensor (np.linspace(tod_max,tod_max,100).reshape(-1, 1), dtype=torch.float32) 
GVD_test_array  = torch.tensor (np.linspace(gvd_fix,gvd_fix,100).reshape(-1, 1), dtype=torch.float32)  #0.1

input_dict = {
    'TOD_fs3': TOD_test_array.squeeze(1).to(dtype=torch.double),
    'GVD': GVD_test_array.squeeze(1).to(dtype=torch.double),
    'z_target_um': z_test_array.squeeze(1).to(dtype=torch.double),
}
output_dict = gp_lume_model.evaluate(input_dict)

In [ ]:
plt.figure()

l1, u1 = (
    output_dict["n_protons_sim_task"].mean.sub(output_dict["n_protons_sim_task"].variance.sqrt().mul_(2)),
    output_dict["n_protons_sim_task"].mean.add(output_dict["n_protons_sim_task"].variance.sqrt().mul_(2)),
)
l2, u2 = (
    output_dict["n_protons_exp_task"].mean.sub(output_dict["n_protons_exp_task"].variance.sqrt().mul_(2)),
    output_dict["n_protons_exp_task"].mean.add(output_dict["n_protons_exp_task"].variance.sqrt().mul_(2)),
)

plt.fill_between(
    z_test_array.squeeze().numpy(), l2.detach().numpy(), u2.detach().numpy(), alpha=0.25, lw=0, color="C0"
)
plt.fill_between(
    z_test_array.squeeze().numpy(), l1.detach().numpy(), u1.detach().numpy(), alpha=0.25, lw=0, color="C1"
)

plt.ylabel("n_protons")

plt.scatter(
    z_test_array.squeeze().numpy(),
    output_dict['n_protons_sim_task'].mean.detach().numpy(),
    color="C1",
    lw=1,
    label="Multi-fidelity GP prediction\n for low-fidelity (sim) data",
)

plt.scatter(
    z_test_array.squeeze().numpy(),
    output_dict['n_protons_exp_task'].mean.detach().numpy(),
    color="C0",
    lw=1,
    label="Multi-fidelity GP prediction\n for high-fidelity (exp) data",
)

plt.legend(loc=0, fontsize="small")
plt.grid()
plt.xlabel('z_target_um')

# Save lume GP model

In [ ]:
# Save the model
gp_lume_model.dump(
    file='./saved_models/' + setup +'.yml',
    save_models=True,    # this saves the underlying botorch/gpytorch model to a .pth file
)

# Created a PhysicalGPModel class to get predictions and uncertainty in physical units

In [ ]:
# Created a PhysicalGPModel Class: 
#    - looading normalization;
#    - predicting with physical units & uncertainty

class PhysicalGPModel:
    def __init__(self, lume_gp, norm_json_path):
        with open(norm_json_path, "r") as f:
            norm = json.load(f)

        self.lume_gp = lume_gp  # Store the full LUME GP model
        self.input_mean = torch.tensor(norm['input_mean'], dtype=torch.float32)
        self.input_std = torch.tensor(norm['input_std'], dtype=torch.float32)
        self.output_mean = torch.tensor(norm['output_mean'], dtype=torch.float32)
        self.output_std = torch.tensor(norm['output_std'], dtype=torch.float32)

    def predict(self, physical_input_dict):
        # Stack input values into a tensor in correct order
        input_tensor = torch.stack([
            physical_input_dict[name] for name in self.lume_gp.input_names
        ], dim=-1)

        # Normalize inputs
        norm_input = (input_tensor - self.input_mean) / self.input_std

        # Get posterior from BoTorch model
        posterior = self.lume_gp.model.posterior(norm_input)

        # Convert normalized outputs back to physical units
        transformed_mean = posterior.mean * self.output_std + self.output_mean
        transformed_variance = posterior.variance * (self.output_std ** 2)

        # Build result dictionary with the same structure
        output_dict = {
            name: torch.distributions.MultivariateNormal(
                loc=transformed_mean[..., i],
                covariance_matrix=torch.diag(transformed_variance[..., i])
            )
            for i, name in enumerate(self.lume_gp.output_names)
        }

        return output_dict

# Load lume GP model and print corresponding predictions and uncertainty in physical units

In [ ]:
loaded_lume_gp_model = GPModel.from_yaml("./saved_models/ip2.yml")

In [ ]:
tod_max = df['TOD_fs3'].max()
zmin, zmax = df['z_target_um'].min(), df['z_target_um'].max()
gvd_fix = (df['GVD'].max()+ df['GVD'].min())/2

z_test_array = torch.tensor (np.linspace(zmin,zmax,100).reshape(-1, 1), dtype=torch.float32) 
TOD_test_array  = torch.tensor (np.linspace(tod_max,tod_max,100).reshape(-1, 1), dtype=torch.float32) 
GVD_test_array  = torch.tensor (np.linspace(gvd_fix,gvd_fix,100).reshape(-1, 1), dtype=torch.float32) 

In [ ]:
input_dict_phys = {
    'TOD_fs3': TOD_test_array.squeeze(1).to(dtype=torch.float32),
    'GVD': GVD_test_array.squeeze(1).to(dtype=torch.double),
    'z_target_um': z_test_array.squeeze(1).to(dtype=torch.float32),
}
model = PhysicalGPModel(loaded_lume_gp_model, "./normalization/normalization.json")
output_dict_phys = model.predict(input_dict_phys)

In [ ]:
plt.figure()

l1, u1 = (
    output_dict_phys['n_protons_sim_task'].mean - 2. * output_dict_phys['n_protons_sim_task'].variance.sqrt(),
    output_dict_phys['n_protons_sim_task'].mean + 2. * output_dict_phys['n_protons_sim_task'].variance.sqrt(),
)
l2, u2 = (
    output_dict_phys['n_protons_exp_task'].mean - 2. * output_dict_phys['n_protons_exp_task'].variance.sqrt(),
    output_dict_phys['n_protons_exp_task'].mean + 2. * output_dict_phys['n_protons_exp_task'].variance.sqrt(),
)

plt.fill_between(
    z_test_array.squeeze().numpy(), l2.detach().numpy(), u2.detach().numpy(), alpha=0.25, lw=0, color="C0"
)
plt.fill_between(
    z_test_array.squeeze().numpy(), l1.detach().numpy(), u1.detach().numpy(), alpha=0.25, lw=0, color="C1"
)

plt.ylabel("n_protons")

plt.scatter(
    z_test_array.squeeze().numpy(),
    output_dict_phys["n_protons_sim_task"].mean.detach().numpy(),
    color="C1",
    lw=1,
    label="Multi-fidelity GP prediction\n for low-fidelity (sim) data",
)

plt.scatter(
    z_test_array.squeeze().numpy(),
    output_dict_phys["n_protons_exp_task"].mean.detach().numpy(),
    color="C0",
    lw=1,
    label="Multi-fidelity GP prediction\n for high-fidelity (exp) data",
)

plt.legend(loc=0, fontsize="small")
plt.grid()
plt.xlabel('z_target_um')